<a href="https://colab.research.google.com/github/mansivyas26/perfusion_dataPipeline/blob/main/variables_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import ast
import warnings
from pandas_gbq import to_gbq
from google.cloud import storage
from google.cloud import bigquery

import pickle

warnings.filterwarnings('ignore')

In [73]:
from google.cloud import storage
import pickle
import pandas as pd

def download_pickle_from_gcs(bucket_name):
    try:
        # Initialize a client
        key_path = '/content/drive/MyDrive/Pefusion_DataPipeline/platform-developement-dev-env-7a2997352c88.json'
        client = storage.Client.from_service_account_json(key_path)

        # Get the bucket
        bucket = client.get_bucket(bucket_name)

        # List all blobs (files) in the bucket
        blobs = bucket.list_blobs()

        # Iterate over each blob
        for blob in blobs:
            # Check if the blob is a file and its name is "all_data.pkl"
            if blob.name.endswith('.pkl') and blob.name.split('/')[-1] == 'all_data.pkl':
                # Download the file
                content = blob.download_as_string()

                # Load the pickle object from the downloaded content
                pickle_object = pickle.loads(content)

                # Convert the pickle object to a DataFrame
                df = pd.DataFrame(pickle_object)

                # Print the DataFrame
                print("DataFrame:")
                print(df)

                # Print success message
                print("Load success!!!")

                return df

        # If no file named "all_data.pkl" is found
        print("File 'all_data.pkl' not found in the bucket.")
        return None

    except Exception as e:
        print(f"Error: {e}")
        return None



In [ ]:
!pip install --upgrade google-cloud-storage


In [74]:
bucket_name = "mark2_runs_bex_ds_dev"
file_name = "all_data.pkl"
pickle_object = download_pickle_from_gcs(bucket_name)


DataFrame:
            time  run_start_flag  pulse_generator_autoregulation_flag  \
0       0.000000             NaN                                  NaN   
1       0.000278             0.0                                  0.0   
2       0.000556             0.0                                  0.0   
3       0.000833             0.0                                  0.0   
4       0.001111             0.0                                  0.0   
...          ...             ...                                  ...   
95525  26.534722             1.0                                  1.0   
95526  26.535000             1.0                                  1.0   
95527  26.535278             1.0                                  1.0   
95528  26.535556             1.0                                  1.0   
95529  26.535833             1.0                                  1.0   

       glucose_control_mode_auto_flag  \
0                                 NaN   
1                             

In [ ]:
#read individual runs file .pkl from bigquery
#perfusion_data = pd.read_pickle("/content/drive/MyDrive/Steve_BEXT84_01-17-2024_13-56-14_all_data.pkl")

In [75]:
perfusion_data = pickle_object

In [76]:
#read variable dictionary
variable_dict = pd.read_csv("/content/drive/MyDrive/Pefusion_DataPipeline/test_262.02 - test.csv")

variable_dict['possible_variables'] = variable_dict['possible_variables'].apply(ast.literal_eval)
#variable_dict.to_csv("/content/drive/MyDrive/Pefusion_DataPipeline/test.csv")


In [77]:
#reading the existing schema
schema = pd.read_csv("/content/drive/MyDrive/Pefusion_DataPipeline/m12_full_join.csv",nrows=10)

In [ ]:
perfusion_data_subset = perfusion_data.head(3)

In [78]:
#check scehma with variable dictionary and renames the column if needed to standard_variable
def check_schema(schema,variable_dict):
  columns = schema.columns.tolist()
  for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
            if column in row['possible_variables']:
                match_found = True
                if column != row['standard_variable']:
                    print("columns that were renamed",column)
                    schema.rename(columns={column: row['standard_variable']}, inplace=True)

In [79]:
#function call
check_schema(schema,variable_dict)

columns that were renamed glucose_control_mode
columns that were renamed syringe_pump_manual_setrate
columns that were renamed time
columns that were renamed probe_ph_mv_measured
columns that were renamed glucose_pump_rate_setpoint


In [80]:

def check_columns(df):
  val = 'None'
  if val in df.columns:
      # Drop the column named 'None'
      df = df.drop(columns=[val])


In [81]:
check_columns(perfusion_data_subset)

In [82]:
check_columns(schema)

In [83]:
#compare .pkl file , variable dicitionary and Schema

def check_runs_with_variable_dict_new(pkl_df, variable_dict, schema_subset):

    removed_columns = []  # List to keep track of REMOVED columns
    list_of_files_ignored_in_pkl = ['timestamp','glucose_syringe_pump_buzzer']
    list_of_files_ignored_in_schema = ['resistance_qc_solenoid_flag','perfusion_date','perfusion_id','control_time','glucose_syringe_pump_dispensed_control', 'glucose_syringe_pump_buzzer_control']

    temporary_none = ['dialysis_compensate_scale_weight','dialysis_flux_valve_position','viscometer_temperature','dialysis_post_pressure','dialysis_compensate_pump_rate','dialysis_compensate_scale_flux','dialysis_pre_pressure','dialysis_pre_pressure_setpoint','dialysis_pressure_valve_position','dialysis_waste_scale_flux','dialysis_waste_scale_weight','']
    #list of variables to be added in schema
    for idx, row in variable_dict.iterrows():
            if row['standard_variable'] in list_of_files_ignored_in_schema:
                  pkl_df[row['standard_variable']] = "to be added"
    columns = pkl_df.columns.tolist()
    for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
           if column in row['possible_variables']:
                match_found = True
                if row['variable_status'] == "EXISTING" and column != row['standard_variable']:
                    pkl_df.rename(columns={column: row['standard_variable']}, inplace=True)
                    if column in schema_subset.columns:
                        schema_subset.rename(columns={column: row['standard_variable']}, inplace=True)
                elif row['variable_status'] == "NEW":
                    #add a column with name row['standard_variable'] and  value = None
                    variable_dict.loc[idx,'variable_status'] = "EXISTING"
                  #rename to standard variable
                    if row['standard_variable'] not in schema_subset.columns:
                        schema_subset[row['standard_variable']] = None

                elif row['variable_status'] == "REMOVED":
                   # print(f"Removed column identified: {column}")
                    removed_columns.append(column)
                    for c in removed_columns:
                      print(removed_columns)
                      pkl_df[c] = None


           if not match_found:
              if row['variable_status'] == "REMOVED":
               pkl_df[column] = "removed"
              for val in temporary_none:
                if val in schema_subset.columns and val not in pkl_df.columns:
        # Add a new column to the pkl DataFrame with the value initialized as None
                 pkl_df[val] = "None"


    cols_to_drop = list(set(pkl_df.columns).intersection(set(list_of_files_ignored_in_pkl)))
    print(cols_to_drop)
    pkl_df = pkl_df.drop(cols_to_drop,inplace=True,axis=1)








In [49]:
def check_columns_match(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    if columns_df1 != columns_df2:
        columns_only_in_df1 = columns_df1 - columns_df2
        columns_only_in_df2 = columns_df2 - columns_df1

        print("Columns only in .pkl file:", columns_only_in_df1)
        print("Columns only in schema:", columns_only_in_df2)
        print(len(columns_only_in_df2),"only_schema")
        print(len(columns_only_in_df1),"only_pkl")

        return False
    else:
        print(len(df1.columns),"pkl")
        print(len(df2.columns),"schema")
        print("Columns match between the two DataFrames.")
        return True

def append_df_row_wise(pkl_df, schema_df):
    #pkl_df = pkl_df.drop(columns=['None'],inplace=True)

    # Check if columns match
    if not check_columns_match(pkl_df, schema_df):
      return None
    else:
      schema_df = schema_df[sorted(schema_df.columns)]
      pkl_df = pkl_df[sorted(pkl_df.columns)]



      #add rows only if perfusion_id looks like BEXT
      combined_df = schema_df.append(pkl_df,ignore_index=True)

      #combined_df.to_csv("/content/drive/MyDrive/Pefusion_DataPipeline/combined.csv")
      print(combined_df)



In [ ]:
#create individual table for dryrun - filter perfusion dataset convert bq table

In [ ]:
#create indivudual table for human runs - filter perfusion dataset convert bq table

In [50]:
print(append_df_row_wise(perfusion_data_subset,schema))

132 pkl
132 schema
Columns match between the two DataFrames.
    air_supply_pressure  arduino_reset_relay  auto_sampler_flag  \
0                   NaN                  NaN                NaN   
1                   NaN                  NaN                NaN   
2                   NaN                  NaN                NaN   
3                   NaN                  NaN                NaN   
4                   NaN                  NaN                NaN   
5                   NaN                  NaN                NaN   
6                   NaN                  NaN                NaN   
7                   NaN                  NaN                NaN   
8                   NaN                  NaN                NaN   
9                   NaN                  NaN                NaN   
10                  NaN                  NaN                NaN   
11                336.0                  0.0               -1.0   
12                339.0                  0.0                1.0   



In [84]:
print(check_runs_with_variable_dict_new(perfusion_data_subset,variable_dict,schema))

['control_time']
['control_time', 'glucose_syringe_pump_buzzer_control']
['control_time', 'glucose_syringe_pump_buzzer_control']
[]
None


In [ ]:
def duplicate_columns_func(df):
  duplicate_columns = df.columns[df.columns.duplicated()]
  print(duplicate_columns)

  # Drop just one duplicate column
  if duplicate_columns.any():
      column_to_drop = duplicate_columns[0]  # Select the first duplicate column to drop
      df.drop(columns=column_to_drop, inplace=True)
      print("Duplicate column dropped:", column_to_drop)
  else:
      print("No duplicate columns found.")

  print("DataFrame after dropping duplicate column:")
  print(df)

In [ ]:
list1 = ['glucose_syringe_pump_volume_setpoint', 'syringe_pump_volume_setpoint', 'glucose_pump_rate_automatic_setpoint']
list2 = schema.columns.to_list()
list3 = perfusion_data_subset.columns.to_list()

In [ ]:
def find_matching_values(list1, list2):
    return list(set(list1) & set(list2))

In [ ]:
find_matching_values(list1,list3)
#find_matching_values(glucose_syringe_pump_volume_setpoint,glucose_syringe_pump_hour_setrate)

['syringe_pump_volume_setpoint', 'glucose_pump_rate_automatic_setpoint']

In [65]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound


def create_table_and_load_df(project_id, dataset_id, table_id, df, key_path):
    # Initialize BigQuery client with service account key
    client = bigquery.Client.from_service_account_json(key_path, project=project_id)
    dataset_ref = client.dataset(dataset_id)

    # Create dataset if it doesn't exist
    dataset = bigquery.Dataset(dataset_ref)
    if not client.get_dataset(dataset_ref, retry=None):
        dataset = client.create_dataset(dataset)

    table_ref = dataset_ref.table(table_id)

    # Define the schema based on the DataFrame data types
    schema = []
    for column, dtype in df.dtypes.items():
        if dtype == 'int64':
            schema.append(bigquery.SchemaField(column, 'INTEGER'))
        elif dtype == 'float64':
            schema.append(bigquery.SchemaField(column, 'FLOAT64'))
        elif dtype == 'object':
            schema.append(bigquery.SchemaField(column, 'STRING'))
        # Add more conditions for other data types as needed

    # Create the table with the specified schema
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)

    # Load the DataFrame into the table
    job_config = bigquery.LoadJobConfig(schema=schema)
    client.load_table_from_dataframe(df, table_ref, job_config=job_config).result()

    print(f'Data loaded into BigQuery table {project_id}.{dataset_id}.{table_id}.')


In [68]:
project_id = 'platform-developement-dev-env'
dataset_id = 'dataConsistency_test'
table_id = 'BEXT84_updated_test'
key_path = "/content/drive/MyDrive/Pefusion_DataPipeline/platform-developement-dev-env-7a2997352c88.json"

# Call the function to create the table (if it doesn't exist) and load the DataFrame
create_table_and_load_df(project_id, dataset_id, table_id, perfusion_data_subset_copy,key_path)

Data loaded into BigQuery table platform-developement-dev-env.dataConsistency_test.BEXT84_updated_test.


In [44]:
from collections import Counter

my_list = perfusion_data_subset.columns.to_list()  # Example list

# Count occurrences of each element
counts = Counter(my_list)

# Find duplicate values
duplicates = [item for item, count in counts.items() if count > 1]

print("Duplicate values:", duplicates)

Duplicate values: ['glucose_syringe_pump_hour_setrate']
